In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import cv2

# Load image
img = cv2.imread('/content/horse.jpeg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title('Original Image')
plt.show()
print(img_rgb.shape)

red, green, blue = cv2.split(img_rgb)

# Normalize the channels
red = red.astype(float) / 255.0
green = green.astype(float) / 255.0
blue = blue.astype(float) / 255.0
blue /= 255.0

# Perform PCA from scratch
class PCAFromScratch:
    def __init__(self, n_components):
        self.n_components = n_components
        self.mean = None
        self.components = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        centered_data = X - self.mean

        cov_matrix = np.cov(centered_data, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        sorted_indices = np.argsort(eigenvalues)[::-1]
        self.components = eigenvectors[:, sorted_indices[:self.n_components]]

    def transform(self, X):
        # Center the input data
        centered_data = X - self.mean
        reduced_data = np.dot(centered_data, self.components)

        return reduced_data

    def inverse_transform(self, X_reduced):
        return np.dot(X_reduced, self.components.T) + self.mean

# Number of components for PCA
n_components = 10

# Create an instance of the PCA class
pca_red = PCAFromScratch(n_components)
pca_green = PCAFromScratch(n_components)
pca_blue = PCAFromScratch(n_components)

# Fit the PCA model to each channel
pca_red.fit(red)
pca_green.fit(green)
pca_blue.fit(blue)

# Transform each channel
red_transformed = pca_red.transform(red)
green_transformed = pca_green.transform(green)
blue_transformed = pca_blue.transform(blue)

# Check the shape of each transformed channel
print(red_transformed.shape, green_transformed.shape, blue_transformed.shape)

# Inverse transform to recover the original data shape
red_reconstructed = pca_red.inverse_transform(red_transformed)
green_reconstructed = pca_green.inverse_transform(green_transformed)
blue_reconstructed = pca_blue.inverse_transform(blue_transformed)

# Check the shape of each reconstructed channel
print(red_reconstructed.shape, green_reconstructed.shape, blue_reconstructed.shape)

# Merge all the inverse transforms into one
merged_reconstructed = cv2.merge([red_reconstructed, green_reconstructed, blue_reconstructed])

# Display the original image and the compressed image obtained after merging
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(np.clip(merged_reconstructed, 0, 1))  # Clip to the valid range [0, 1]
plt.title('Compressed Image')

plt.show()


ModuleNotFoundError: No module named 'cv2'

Check the shape of image